In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [ ]:
#As data size is huge we will be dealing with a million rows only
%time
df_train =  pd.read_csv('../input/train.csv', nrows = 2_000_000, parse_dates=["pickup_datetime"])

# list first few rows (datapoints)
df_train.head()

In [ ]:
df_train.describe()

As count is different for columns ,hence removing null values(only for now).

In [ ]:
df_train = df_train.dropna(how = 'any', axis = 'rows')
df_train.describe()


**UNIVARIATE ANALYSIS**


**1)fare_amount**
It is an ordinal variable.
Removing 1) negative values,2)By checking Newyorktaxi fare system no fare can be around 1200 ,and  

In [ ]:
df_train = df_train.drop(df_train[df_train['fare_amount']<0].index, axis=0)
df_train.shape

In [ ]:
df_train['fare_amount'].value_counts().sort_index().plot.line(xlim=(0,1300))

In [ ]:
df_train['fare_amount'].describe()

In [ ]:
df_train[df_train.fare_amount<100].fare_amount.hist(bins=100, figsize=(14,3))

http://nymag.com/nymetro/urban/features/taxi/n_20286/ Checking fares from this website ,we can see the minimum fare is 2.5 and taking 45 as maximum fare ,we can safely conclude no fare will be more than 100,so dropping all values which donot lie between 2.5 and 100
Also we conclude some permanent fares between 40-60 (Newark Airport=$45)

In [ ]:
df_train = df_train[(df_train.fare_amount>=2.5) & (df_train.fare_amount<=100) ]
df_train['fare_amount'].describe()

**2)passenger_count**

In [ ]:
df_train['passenger_count'].describe()

Clearly 208 is an outliear !,Considering a passenger_count can be between 1 and 7 both included(an SUV with capacity 7) ,removing every other value. 
#Can be done (Optional)
Currently we are removing 0 passenger _count entries  from a training set, later we can add them by applying linear regression to these values.

In [ ]:
df_train = df_train[(df_train.passenger_count < 7) & (df_train.passenger_count>0 )]
df_train['passenger_count'].value_counts().sort_index().plot.bar()


**3)Geospatial Analysis**

Using haversine formula

By using google we found coordinates of New york city to be from -90 to 90,-180 to 180

In [ ]:
df_train = df_train.drop(((df_train[df_train['dropoff_latitude']<-90])|(df_train[df_train['dropoff_latitude']>90])).index, axis=0)
df_train[df_train['dropoff_latitude']<-180]|df_train[df_train['dropoff_latitude']>180]


In [ ]:
def haversine_distance(lat1, long1, lat2, long2):
    data = [df_train]
    for i in data:
        R = 6371  #radius of earth in kilometers
        #R = 3959 #radius of earth in miles
        phi1 = np.radians(i[lat1])
        phi2 = np.radians(i[lat2])
    
        delta_phi = np.radians(i[lat2]-i[lat1])
        delta_lambda = np.radians(i[long2]-i[long1])
    
        #a = sin²((φB - φA)/2) + cos φA . cos φB . sin²((λB - λA)/2)
        a = np.sin(delta_phi / 2.0) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2.0) ** 2
    
        #c = 2 * atan2( √a, √(1−a) )
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
        #d = R*c
        d = (R * c) #in kilometers
        i['H_Distance'] = d
    return d

In [ ]:
haversine_distance('pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude')

**Preparing date columns **

In [ ]:
data = [df_train]
for i in data:
    i['Year'] = i['pickup_datetime'].dt.year
    i['Month'] = i['pickup_datetime'].dt.month
    i['Date'] = i['pickup_datetime'].dt.day
    i['Day of Week'] = i['pickup_datetime'].dt.dayofweek
    i['Hour'] = i['pickup_datetime'].dt.hour

In [ ]:
df_train.head()

**BIVARIATE ANALYSIS**

 Number of passengers and the fare_count

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(df_train['passenger_count'], bins=15)
plt.xlabel('No. of Passengers')
plt.ylabel('Frequency')

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(x=df_train['passenger_count'], y=df_train['fare_amount'], s=1.5)
plt.xlabel('No. of Passengers')
plt.ylabel('Fare')

We can learn from above data they are correlated ,as most of the fare comes from passenger_count == 1

 **date pickup and fare?**

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(df_train['Date'], y=df_train['fare_amount'], s=1.4)
plt.xlabel('Date')
plt.ylabel('Fare')

Not much can be concluded from this graph,it is mostly uniform

**Day pickup and fare**

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(df_train['Day of Week'], bins=100)
plt.xlabel('Day of Week')
plt.ylabel('Frequency')

Not much can be concluded from this graph,as most people travel during weekend.

**time of pickup and  fare?**

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(df_train['Hour'], bins=100)
plt.xlabel('Hour')
plt.ylabel('Frequency')

We can conclude that  frequency of taxi_rides at 5am is low and around 8pm its high .

In [ ]:
plt.figure(figsize=(15,7))
plt.scatter(x=df_train['Hour'], y=df_train['fare_amount'], s=1.5)
plt.xlabel('Hour')
plt.ylabel('Fare')

#NEEDS TO BE INVESTIGATED
The fare is high around 5am and 2pm,even though frequency of travelling is low

**Distance and Fare amount**

In [ ]:
df_train.sort_values(['H_Distance','fare_amount'], ascending=False)

In [ ]:
bins_0 = df_train.loc[( df_train['H_Distance'] == 0), ['H_Distance']]
bins_1 =  df_train.loc[( df_train['H_Distance'] > 0) & ( df_train['H_Distance'] <= 10),['H_Distance']]
bins_2 =  df_train.loc[( df_train['H_Distance'] > 10) & ( df_train['H_Distance'] <= 50),['H_Distance']]
bins_3 =  df_train.loc[( df_train['H_Distance'] > 50) & ( df_train['H_Distance'] <= 100),['H_Distance']]
bins_4 =  df_train.loc[( df_train['H_Distance'] > 100) & ( df_train['H_Distance'] <= 200),['H_Distance']]
bins_5 =  df_train.loc[( df_train['H_Distance'] > 200) & ( df_train['H_Distance'] <= 300),['H_Distance']]
bins_6 =  df_train.loc[( df_train['H_Distance'] > 300),['H_Distance']]
bins_0['bins'] = '0'
bins_1['bins'] = '0-10'
bins_2['bins'] = '11-50'
bins_3['bins'] = '51-100'
bins_4['bins'] = '100-200'
bins_5['bins'] = '201-300'
bins_6['bins'] = '>300'
dist_bins =pd.concat([bins_0,bins_1,bins_2,bins_3,bins_4,bins_5,bins_6])
#len(dist_bins)
dist_bins.columns

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(dist_bins['bins'], bins=75)
plt.xlabel('Bins')
plt.ylabel('Frequency')

In [ ]:
df_train = df_train.drop(df_train.loc[((df_train['pickup_latitude']==0) & (df_train['pickup_longitude']==0))&((train['dropoff_latitude']!=0) & (df_train['dropoff_longitude']!=0)) & (df_train['fare_amount']==0)].index, axis=0)

In [ ]:
df_train = df_train.drop(df_train.loc[((df_train['pickup_latitude']!=0) & (df_train['pickup_longitude']!=0))&((df_train['dropoff_latitude']==0) & (df_train['dropoff_longitude']==0)) & (df_train['fare_amount']==0)].index, axis=0)